In [9]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
from pathlib import Path
from pprint import pprint
import urltools
import numpy as np
import csv

import matplotlib.pyplot as plt 
import seaborn as sns

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

import configparser

from _helpers import *

In [10]:
pd.set_option("display.precision", 2)
sns.set_style("darkgrid")

In [11]:
root = Path('../../')
Config = configparser.ConfigParser()
Config.read(str(root / 'config.cnf'))

# Load files from disk
queries = root / Config.get('input_files', 'queries')
queries = load_queries(str(queries))

am = root / Config.get('input_files', 'altmetric')
am = pd.read_csv(str(am))
# am_ = pd.read_excel("../../data/input/NewsMentions-AltmetricOct2017.xlsx", sheet_name=1)

# tweet_files = root / Config.get('output_files', 'tweets')

twitter_urls = root / Config.get('output_files', 'twitter_urls')
twitter_urls = list(twitter_urls.glob("*.csv"))

summary_tweets = root / Config.get('output_files', 'summary_tweets')
summary_tweets = pd.read_csv(summary_tweets)
summary_tweets.set_index("venue", inplace=True)

summary_urls = root / Config.get('output_files', 'summary_urls')

## Errors

In [5]:
exp = pd.read_csv("temp/expanded_urls.csv", index_col=0, parse_dates=['timestamp'])

/home/asura/.virtualenvs/altmetrics/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Work with final dataset

In [12]:
dfs = []
for file in tqdm(twitter_urls):
    df = load_urls(file)
    
    df['venue'] = file.name.split("/")[-1].split(" ")[0]
    dfs.append(df)
urls = pd.concat(dfs)
urls.set_index("Unnamed: 0", inplace=True)
urls.index.name = "id"
urls.head()

,tweet_id,relevant_url,expanded,timestamp,retweeted_status,quoted_status,cleaned_url,relevant,venue
id,,,,,,,,,
0,786920087324352512,http://www.slate.com/blogs/bad_astronomy/2016/...,False,2018-11-23 07:37:30.366753,NaN,NaN,www.slate.com/blogs/bad_astronomy/2016/10/14/t...,True,slate
1,788006486777749504,http://www.slate.com/blogs/bad_astronomy/2016/...,False,2018-11-23 07:37:30.366969,NaN,NaN,www.slate.com/blogs/bad_astronomy/2016/10/17/o...,True,slate
2,788368907719569408,http://www.slate.com/blogs/bad_astronomy/2016/...,False,2018-11-23 07:37:30.367317,NaN,NaN,www.slate.com/blogs/bad_astronomy/2016/10/18/m...,True,slate
3,788734240649273344,http://www.slate.com/blogs/bad_astronomy/2016/...,False,2018-11-23 07:37:30.367608,NaN,NaN,www.slate.com/blogs/bad_astronomy/2016/10/19/h...,True,slate
4,789094121935953920,http://www.slate.com/blogs/bad_astronomy/2016/...,False,2018-11-23 07:37:30.367905,NaN,NaN,www.slate.com/blogs/bad_astronomy/2016/10/20/a...,True,slate


In [13]:
summary_tweets['found tweets'].sum(), urls.relevant_url.count(), urls.cleaned_url.nunique(), 100*urls.relevant.sum()/summary_tweets['found tweets'].sum()

(3584984, 1607182, 18062, 44.71735438707676)

In [14]:
summary_tweets['found urls'] = urls.venue.value_counts()
summary_tweets['relevant urls'] = urls.groupby("venue")['relevant'].sum()
summary_tweets['unique urls'] = urls.groupby("venue")['cleaned_url'].nunique()

x = summary_tweets[['found tweets', 'found urls', 'relevant urls', 'unique urls']]

x[['relevant urls']] = x[['relevant urls']].div(x['found tweets']/100, axis="rows")

x.columns = ['found tweets', 'found urls', 'relevant urls [%]', 'unique urls']
x = x.round(1).replace(np.nan, 0)
x.to_csv(summary_urls)
x

/home/asura/.virtualenvs/altmetrics/lib/python3.5/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,found tweets,found urls,relevant urls [%],unique urls
venue,,,,
bostonglobe,16468,4182.0,25.4,226.0
chicago,830,0.0,0.0,0.0
foxnews,229911,149869.0,65.2,3475.0
iflscience,184242,0.0,0.0,0.0
latimes,125809,84034.0,66.8,2066.0
nytimes,1124602,686997.0,60.9,4569.0
sfchronicle,9863,4984.0,50.5,325.0
slate,1873,646.0,34.5,219.0
theglobeandmail,26130,4897.0,18.7,176.0


In [84]:
summary_tweets

,found tweets,original,retweets,quotes,retweets/quotes,reply,found urls,relevant urls,unique urls
venue,,,,,,,,,
bostonglobe,16468,3996,11616,317,539,129,4182.0,4182.0,226.0
chicago,830,265,564,1,0,4,NaN,NaN,NaN
foxnews,229911,100453,88014,2032,39412,2363,149869.0,149868.0,3475.0
iflscience,184242,128366,53984,1488,404,2947,NaN,NaN,NaN
latimes,125809,53174,71081,514,1040,1088,84034.0,84031.0,2066.0
nytimes,1124602,422636,687368,5432,9166,6588,686997.0,684332.0,4569.0
sfchronicle,9863,2822,6970,39,32,40,4984.0,4984.0,325.0
slate,1873,876,942,15,40,21,646.0,646.0,219.0
theglobeandmail,26130,8034,17999,45,52,583,4897.0,4897.0,176.0
